# Semantic Segmentation
- Model : yolov8s-seg
- train image size : 640x640


## Data Pipeline
- google drive에서 colab storage로 복사
- 압축파일 해제 후 각 폴더별 xml 파일기반으로 label txt 생성


In [1]:
import cv2
import xml.etree.ElementTree as ET
import numpy as np
from tqdm import tqdm
import shutil
import zipfile, os
import yaml
import pandas as pd

#### 1) 파일 경로 가져오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# goolge drive에 파일 저장된 경로
gdrive_path = '/content/drive/MyDrive/인도보행 영상/서피스마스킹/'

# 압축 파일명 전부 입력
file_names = ['Surface_1.zip','Surface_2.zip','Surface_3.zip','Surface_4.zip','Surface_5.zip']

# # 파일 복사해오기
# for file_name in file_names:
#     shutil.copy(gdrive_path + file_name, '/content/' + file_name)

#### 2) 압축 해제 및 colab storage 저장

In [4]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()
        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return
        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./data/')

# 압축해제후 저장될 폴더 생성
!mkdir /content/data

# 압축 해제
for file_name in file_names:
    dataset_extract(gdrive_path + file_name)

Extracting: 100%|██████████| 18046/18046 [02:02<00:00, 147.07files/s]


#### 3) label txt 생성

In [5]:
# class 별 숫자 지정
classes = {'sidewalk':0, # 인도
           'braille_guide_blocks' : 1, # 점자블록
           'damaged_braille_guide_blocks' : 2, # 손상된 점자블록
           'roadway' : 3, # 차도
           'alley' : 4, # 이면도로
           'bike_lane' : 5, # 자전거도로
           # caution_zone
           'grating' : 6, # 그레이팅
           'manhole' : 7, # 맨홀
           'repair_zone' : 8, # 보수구역
           'stairs' : 9, # 계단
           'tree_zone' : 10, # 가로수 영역
           # crosswalk
           'alley_crosswalk' : 11, # 이면도로의 횡단보도
           'roadway_crosswalk' : 12 # # 차도의 횡단보도
           }

class_img = {'sidewalk':[], # 인도
           'braille_guide_blocks' : [], # 점자블록
           'damaged_braille_guide_blocks' : [], # 손상된 점자블록
           'roadway' : [], # 차도
           'alley' : [], # 이면도로
           'bike_lane' : [], # 자전거도로
           # caution_zone
           'grating' : [], # 그레이팅
           'manhole' : [], # 맨홀
           'repair_zone' : [], # 보수구역
           'stairs' : [], # 계단
           'tree_zone' : [], # 가로수 영역
           # crosswalk
           'alley_crosswalk' : [], # 이면도로의 횡단보도
           'roadway_crosswalk' : [] # # 차도의 횡단보도
           }

def append_class_image(class_list, img_name):
    a = set(class_list) | set([img_name])
    return list(a)


In [6]:
folders = os.listdir('/content/data/')

# 모든 이미지 파일 경로 저장
all_image_pathes = []
all_xml_pathes = []


for folder in folders:
    folder_path = '/content/data/'+folder+'/'
    files = os.listdir(folder_path)
    image_pathes = [folder_path + img_file for img_file in files if img_file[-3:] == 'jpg']
    all_image_pathes = all_image_pathes + image_pathes
    xml_path = [folder_path + xml_file for xml_file in files if xml_file[-3:] == 'xml']
    all_xml_pathes = all_xml_pathes + xml_path

In [7]:
# masking 이미지 저장소 생성
!mkdir /content/masking


# label txt 생성
for xml_path in tqdm(all_xml_pathes):

    # XML 파일 파싱
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for image in root.findall('.//image'):
        line = ""

        height, width = int(image.get('height')), int(image.get('width'))
        for polygon in image.findall('.//polygon'):
            # label 값에 대응하는 숫자 가져오기
            cls = polygon.get('label')

                # 점자블록 파손여부 구별하기
            if cls == 'braille_guide_blocks':
                if polygon.findall('.//attribute')[0].text == 'damaged':
                    cls = 'damaged_braille_guide_blocks'

                # 주의구역 속성 구분하기
            elif cls == 'caution_zone':
                cls = polygon.findall('.//attribute')[0].text
                # alley의 횡단보도 구분하기
            elif cls == 'alley':
                if len(polygon.findall('.//attribute')) > 0:
                    if polygon.findall('.//attribute')[0].text == 'crosswalk':
                        cls = 'alley_crosswalk'
                # roadway의 횡단보도 구분하기
            elif cls == 'roadway':
                if polygon.findall('.//attribute')[0].text == 'crosswalk':
                    cls = 'roadway_crosswalk'

            class_img[cls] = append_class_image(class_img[cls], image.get('name'))
            label = classes[cls]
            line += str(label)
            # points 값 가져오기
            points_str = polygon.get('points')
            # points를 저장하기
            points = []
            for point_str in points_str.split(';'):
                x, y = map(float, point_str.split(','))
                line += f" {str(round(x/width,6))} {str(round(y/height,6))}"
            line += "\n"

        with open('/content/masking/'+image.get('name').replace('.jpg','.txt'), 'w') as file:
            file.writelines(line)

100%|██████████| 800/800 [03:38<00:00,  3.67it/s]


#### ※ 다운샘플링


In [8]:
# 각 클래스를 포함하는 이미지의 수
for key in class_img.keys():
    print(key,':',len(class_img[key]))

sidewalk : 34116
braille_guide_blocks : 8126
damaged_braille_guide_blocks : 160
roadway : 15491
alley : 22907
bike_lane : 3092
grating : 11257
manhole : 14062
repair_zone : 172
stairs : 1390
tree_zone : 7699
alley_crosswalk : 1211
roadway_crosswalk : 2328


- 현재 dataset에서 class를 제대로 인식하려면 최소 1000장 이상의 이미지가 필요하다고 판단 됨
- 가설 : 수가 적은 클래스를 포함하는 이미지는 제거하지 않고 나머지 이미지의 수를 1/10로 줄인다면 클래스를 포함하는 이미지가 100장 이상만 되어도 유의미해 질 것이다

In [9]:
# 전체이미지
all_img = []
for key in class_img.keys():
   all_img = list(set(all_img) | set(class_img[key]))

# 수가 적은 클래스를 포함하는 이미지
few_class_img = list(set(class_img['damaged_braille_guide_blocks']) | set(class_img['repair_zone']) | set(class_img['stairs']) |
                    set(class_img['alley_crosswalk']) | set(class_img['roadway_crosswalk']) | set(class_img['bike_lane']))
# few_class_img = list(set(class_img['damaged_braille_guide_blocks']))

# 다운샘플링을 적용할 이미지
down_image = list(set(all_img) - set(few_class_img))

In [10]:
print(len(all_img))
print(len(few_class_img))
print(len(down_image))

46396
7693
38703


In [11]:
from sklearn.model_selection import train_test_split
_, use_image, _, _ = train_test_split(down_image, down_image, test_size=0.20, random_state=42)
use_image = use_image + few_class_img
len(use_image)

15434

In [12]:
# 다운샘플링 후 각 클래스를 포함하는 이미지의 수
for key in class_img.keys():
    print(key,':',len(set(class_img[key])&set(use_image)))

sidewalk : 12398
braille_guide_blocks : 4254
damaged_braille_guide_blocks : 160
roadway : 6589
alley : 6258
bike_lane : 3092
grating : 3373
manhole : 5093
repair_zone : 172
stairs : 1390
tree_zone : 2991
alley_crosswalk : 1211
roadway_crosswalk : 2328


#### 3) yolo가 요구하는 폴더 구조로 위에서 선택된 파일들만 이동

In [13]:
import os
# 폴더구조 생성
os.makedirs('/content/datasets/images/train', exist_ok=True)
os.makedirs('/content/datasets/images/val', exist_ok=True)
os.makedirs('/content/datasets/labels/train', exist_ok=True)
os.makedirs('/content/datasets/labels/val', exist_ok=True)

In [14]:
# train val 폴더로 이동시키기

from sklearn.model_selection import train_test_split

use_image_pathes = []
for img_path in all_image_pathes:
    if img_path[(img_path.rfind('/')+1):] in use_image:
        use_image_pathes.append(img_path)

train_image_path, val_image_path, _, _ = train_test_split(use_image_pathes,use_image_pathes, test_size=0.35, random_state=42)

for img_path in tqdm(train_image_path):
    file_name = img_path[img_path.rfind('/'):]
    # 위에서 선택한 use_file에 해당된다면 이동
    shutil.copy(img_path, '/content/datasets/images/train'+file_name)

for img_path in tqdm(val_image_path):
    file_name = img_path[img_path.rfind('/'):]
    shutil.copy(img_path, '/content/datasets/images/val'+file_name)

100%|██████████| 5402/5402 [00:51<00:00, 104.57it/s]


In [15]:
train_mask_path = [file_name.replace('jpg','txt') for file_name in os.listdir('/content/datasets/images/train')]
val_mask_path = [file_name.replace('jpg','txt') for file_name in os.listdir('/content/datasets/images/val')]

for mask_path in tqdm(train_mask_path):
    shutil.copy('/content/masking/'+mask_path, '/content/datasets/labels/train/'+mask_path)

for mask_path in tqdm(val_mask_path):
    shutil.copy('/content/masking/'+mask_path, '/content/datasets/labels/val/'+mask_path)

mkdir: cannot create directory ‘/content/datasets/labels’: File exists
mkdir: cannot create directory ‘/content/datasets/labels/train’: File exists
mkdir: cannot create directory ‘/content/datasets/labels/val’: File exists


100%|██████████| 5402/5402 [00:00<00:00, 7918.19it/s]


#### ※ 업샘플링
- 파손블록을 포함한 이미지를 복사해 업샘플링

In [16]:
import os
# 폴더구조 생성
os.makedirs('/content/datasets/images/train', exist_ok=True)
os.makedirs('/content/datasets/images/val', exist_ok=True)
os.makedirs('/content/datasets/labels/train', exist_ok=True)
os.makedirs('/content/datasets/labels/val', exist_ok=True)

In [17]:
c = 1
for img_path in all_image_pathes:
    if img_path[(img_path.rfind('/')+1):] in class_img['damaged_braille_guide_blocks']:
        shutil.copy(img_path, '/content/datasets/images/train/'+f'copy{c}.jpg')
        shutil.copy('/content/masking/'+ img_path[(img_path.rfind('/')+1):][:(img_path[(img_path.rfind('/')+1):].rfind('.')+1)] + 'txt',
                    '/content/datasets/labels/train/'+f'copy{c}.txt')
        c += 1

#### 4) 추가데이터 가져오기(로보플로우 데이터 다운로드)

In [18]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZYen26siHcBua4CPijvq")
project = rf.workspace("test-d2wwg").project("seedo")
version = project.version(14)
dataset = version.download("coco-segmentation")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Seedo-프로젝트-14 in coco-segmentation:: 100%|██████████| 327/327 [00:00<00:00, 1349.61it/s]


- coco_dataset 기준 데이터를 yolo 가 형식대로 변환


In [19]:
import json
import os
import shutil
from tqdm import tqdm


# 폴더구조 생성
os.makedirs('/content/datasets/images/train', exist_ok=True)
os.makedirs('/content/datasets/images/val', exist_ok=True)
os.makedirs('/content/datasets/labels/train', exist_ok=True)
os.makedirs('/content/datasets/labels/val', exist_ok=True)

In [20]:
# yolo가 요구하는 txt를 만드는 함수
def cocojson_to_yolosegtxt(json_path, save_path, re_class):
    import json
    from tqdm import tqdm
    # JSON 파일 읽기
    with open(json_path, 'r') as file:
        data = json.load(file)

    if save_path[-1] != '/':
        save_path = save_path+'/'

    # 세그멘테이션 데이터 처리
    segmentation_data = {}
    for i in range(len(data['images'])):
        segmentation_data[str(i)] = []
    for anno in data['annotations']:
       segmentation_data[str(anno['image_id'])].append({'category_id': str(anno['category_id']), 'segmentation': anno['segmentation']})

    for image in tqdm(data['images']):
        line = ""
        for segment in segmentation_data[str(image['id'])]:
            cls = str(re_class[segment['category_id']])
            for segmentation in segment['segmentation']:
                line = line + cls
                w = True
                for s in segmentation:
                    if w:
                        wh = image['width']
                    else:
                        wh = image['height']
                    w = not w
                    line = line + ' ' + str(round(s / wh, 5))
                line = line + '\n'
        file_name = image['file_name']

        with open(save_path + file_name[:file_name.rfind('.'):]+'.txt' , 'w') as file:
            file.writelines(line)

- train

In [21]:
# JSON 파일 경로
file_path = '/content/Seedo-프로젝트-14/train/'


# JSON 파일 읽기
with open(file_path + '_annotations.coco.json', 'r') as file:
    data = json.load(file)

# name을 보고 우리가 원하는 lable로 맞춰주기
data['categories']

classes = {'sidewalk':0, # 인도
           'braille_guide_blocks' : 1, # 점자블록
           'damaged_braille_guide_blocks' : 2, # 손상된 점자블록
           'roadway' : 3, # 차도
           'alley' : 4, # 이면도로
           'bike_lane' : 5, # 자전거도로
           # caution_zone
           'grating' : 6, # 그레이팅
           'manhole' : 7, # 맨홀
           'repair_zone' : 8, # 보수구역
           'stairs' : 9, # 계단
           'tree_zone' : 10, # 가로수 영역
           # crosswalk
           'alley_crosswalk' : 11, # 이면도로의 횡단보도
           'roadway_crosswalk' : 12 # # 차도의 횡단보도
           }

re_class ={}

for cat in data['categories']:
    re_class[str(cat['id'])] = classes[cat['name']]

# 클래스 확인
print(re_class)

# 라벨생성
cocojson_to_yolosegtxt(file_path + '_annotations.coco.json', '/content/datasets/labels/train', re_class)
# 이미지 이동
images = [img for img in os.listdir(file_path) if img[-4:] != 'json']
for img in tqdm(images):
    shutil.move(file_path + img, '/content/datasets/images/train/' + img)

{'0': 0, '1': 4, '2': 11, '3': 5, '4': 1, '5': 2, '6': 6, '7': 7, '8': 8, '9': 3, '10': 12, '11': 0, '12': 9, '13': 10}


100%|██████████| 300/300 [00:00<00:00, 38882.95it/s]


- validation

In [22]:
# JSON 파일 경로
file_path = '/content/Seedo-프로젝트-14/valid/'


# JSON 파일 읽기
with open(file_path + '_annotations.coco.json', 'r') as file:
    data = json.load(file)

# re_class 는 위에서 생성 했으므로 생략

# 라벨생성
cocojson_to_yolosegtxt(file_path + '_annotations.coco.json', '/content/datasets/labels/val', re_class)
# 이미지 이동
images = [img for img in os.listdir(file_path) if img[-4:] != 'json']
for img in tqdm(images):
    shutil.move(file_path + img, '/content/datasets/images/val/' + img)

100%|██████████| 21/21 [00:00<00:00, 16393.15it/s]


#### 5) yaml 생성

In [23]:
path = '/content/datasets'
train_path = '/content/datasets/images/train'
val_path = '/content/datasets/images/val'

class_names = {
    0: 'sidewalk',
    1: 'braille_guide_blocks',
    2: 'damaged_braille_guide_blocks',
    3: 'roadway',
    4: 'alley',
    5: 'bike_lane',
    6: 'grating',
    7: 'manhole',
    8: 'repair_zone',
    9: 'stairs',
    10: 'tree_zone',
    11: 'alley_crosswalk',
    12: 'roadway_crosswalk',}

data = {
    'path': path,
    'train': train_path,
    'val': val_path,
    'names': class_names
}

# data.yaml 파일 생성 및 작성
with open('/content/datasets/data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

## Modeling

#### 모델 정의

In [24]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.7/802.7 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [25]:
from ultralytics import YOLO

model = YOLO("yolov8s-seg.pt")
# model = YOLO("/content/drive/MyDrive/빅프로젝트/모델/yolov8_seg_add_att/epoch50.pt")

100%|██████████| 22.8M/22.8M [00:00<00:00, 226MB/s]


#### Train

In [26]:
model.train(
    data='/content/datasets/data.yaml',
    epochs=300,
    patience=10,
    # resume=True,
    verbose=True,
    pretrained=True,
    seed=503,
    save_period=5,
    lr0=0.01,
    lrf=0.001,
    # perspective= 0.0002,
    # mixup=0.1,
    # copy_paste=0.1,
    # hsv_h=0.02,
)

Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/datasets/data.yaml, epochs=300, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=503, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 22.5MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 6.25M/6.25M [00:00<00:00, 135MB/s]
Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)


AMP: checks passed ✅


...

    102/300       5.4G     0.5225     0.9922     0.6166     0.9588        128        640: 100%|██████████| 656/656 [04:18<00:00,  2.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 170/170 [01:00<00:00,  2.81it/s]


                   all       5423      27176      0.757      0.622      0.678       0.55      0.751      0.615      0.666      0.497
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 92, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

102 epochs completed in 9.053 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 23.9MB
Optimizer stripped from runs/segment/train/weights/best.pt, 23.9MB

Validating runs/segment/train/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,784,631 parameters, 0 gradients, 42.5 GFLOPs


Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 170/170 [01:03<00:00,  2.69it/s]


                   all       5423      27176      0.754      0.616      0.679      0.551      0.752      0.607      0.667      0.498
              sidewalk       4375       7771      0.807      0.707      0.781      0.678      0.805      0.699      0.765      0.623
  braille_guide_blocks       1502       2203      0.859      0.803      0.861      0.749      0.866        0.8      0.855      0.702
damaged_braille_guide_blocks         72        120      0.661      0.392      0.466      0.313      0.677      0.392      0.461      0.285
               roadway       2325       4405      0.746      0.717      0.756      0.621      0.744      0.704      0.742      0.551
                 alley       2158       3248       0.74       0.58      0.692       0.62      0.745      0.576      0.685      0.585
             bike_lane       1077       1257      0.785      0.733      0.807       0.72      0.783      0.729        0.8      0.657
               grating       1162       1460      0.811      0.

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a5befe685b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    

In [27]:
# best모델 gdrive로 복사
os.makedirs('/content/drive/MyDrive/빅프로젝트/모델/yolov8_seg_add_att_업다운샘플링/', exist_ok=True)
shutil.copy('/content/runs/segment/train/weights/best.pt',
            '/content/drive/MyDrive/빅프로젝트/모델/yolov8_seg_add_att_업다운샘플링/no_arg_best_up2dowen20%.pt')

'/content/drive/MyDrive/빅프로젝트/모델/yolov8_seg_add_att_업다운샘플링/best_up2dowen20%.pt'

## 성능 확인

```
class_names = {
    0: 'sidewalk', # 인도
    1: 'braille_guide_blocks', # 점자블록
    2: 'damaged_braille_guide_blocks', #손상된 점자블록
    3: 'roadway', # 차도
    4: 'alley', # 이면도로
    5: 'bike_lane', # 자전거 도로
    6: 'grating', # 그레이팅
    7: 'manhole', # 맨홀
    8: 'repair_zone', # 보수구역
    9: 'stairs', # 계단
    10: 'tree_zone', # 가로수 구역
    11: 'alley_crosswalk', # 이면도로의 횡단보도
    12: 'roadway_crosswalk', # 차도의 횡단보도
}
```



In [ ]:
!pip install ultralytics

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
from ultralytics import YOLO

loaded_model = YOLO('/content/drive/MyDrive/빅프로젝트/모델/yolo_v8_seg/yolov8n_seg16.pt')


results = loaded_model.predict(source='/content/KakaoTalk_20240625_161714222.jpg',
                        save=True,
                        line_width=2)